In [1]:
import pickle
X_file = "/kaggle/input/hardata1/x_data.pickle"
Y_file = "/kaggle/input/hardata1/y_data.pickle"

with open(X_file, 'rb') as file:
    X = pickle.load(file)
with open(Y_file, 'rb') as file:
    Y = pickle.load(file)

In [2]:
from keras.utils import to_categorical
import numpy as np
Y = np.argmax(Y, axis=1)
Y = to_categorical(Y, num_classes=9)

2024-02-16 14:08:00.245889: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 14:08:00.245941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 14:08:00.247435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(np.append(X[:,:,0], X[:,:,1], axis=0))

X_real = scaler.transform(X[:,:,0])
X_imag = scaler.transform(X[:,:,1])

In [4]:
X = np.stack((X_real, X_imag), axis=-1)

In [5]:
from sklearn.model_selection import train_test_split
input_train, input_test, target_train , target_test =  train_test_split(X, Y, test_size=0.2, random_state=45)

In [6]:
del X,Y,X_real,X_imag

In [7]:
from keras.datasets import mnist
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

# Model configuration
batch_size = 250
no_epochs = 5
no_classes = 9
validation_split = 0.2
verbosity = 1


# Shape of the input sets
input_train_shape = input_train.shape
input_test_shape = input_test.shape
print(input_train_shape,input_test_shape )

(1945762, 90, 2) (486441, 90, 2)


In [8]:
# Keras layer input shape
input_shape = (input_train_shape[1], input_train_shape[2], 1)
# Reshape the training data to include channels
input_train = input_train.reshape(input_train_shape[0], input_train_shape[1], input_train_shape[2], 1)
input_test = input_test.reshape(input_test_shape[0], input_test_shape[1], input_test_shape[2], 1)
# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')
# Normalize input data
#input_train = input_train / 127
#input_test = input_test / 127


In [9]:
#Import Libraries
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import  TimeDistributed, Flatten
from tensorflow.keras.layers import LSTM

In [10]:
batch_size=256
num_classes=9
epochs=20

def build_model(optimizer):
    model = Sequential()

    model.add(Conv2D(32, (2, 1), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 1)))
    model.add(Conv2D(64, (2, 1), activation='relu'))
    model.add(MaxPooling2D((2, 1)))
    model.add(TimeDistributed(Flatten()))
    #LSTM layers
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer= optimizer, metrics=['accuracy'])
    return model

In [ ]:




optimizers = ['Adagrad', 'Adam', 'RMSprop', 'SGD']
for i in optimizers:
    model = build_model(i)
    hist=model.fit(input_train,target_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(input_test,target_test))

import pandas as pd
hist_df = pd.DataFrame(hist.history)
# or save to csv:
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
#plotting
from matplotlib import pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()